In [13]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf

import datetime
import time
from datetime import timedelta
import os

%matplotlib inline

In [14]:
df = pd.read_csv(r"C:\Users\rg654th\Downloads\Flight Delay\Clean Data\Clean Historical Data.csv", parse_dates=[0], infer_datetime_format=True)

In [15]:
# New Column and Format
df['Scheduled DateTime'] = pd.to_datetime(df['Scheduled DateTime'], infer_datetime_format=True)
df['Delay? (15 min)'] = np.where(df['Minute Delay'] >= 15, 'Yes', 'No')

In [16]:
# df['Sched_mins_since_midnight']
df['Sched_mins_since_midnight'] = (df['Scheduled DateTime'] - df['Scheduled DateTime'].apply(lambda dt: dt.replace(hour=0, minute=0, second=0, microsecond=0)))
df['Sched_mins_since_midnight'] = df['Sched_mins_since_midnight'] / np.timedelta64(1, 'm')

In [17]:
traindf = df.drop(['Flight No.', 'Scheduled Date','Scheduled DateTime', 'Scheduled Time', 'Real Departure Time'], axis=1)
traindf = traindf.dropna()
traindf.shape
traindf.head(1)

,Terminal,Destination,Airline,Month,Weekday,Minute Delay,Delay Type,Delay? (15 min),Sched_mins_since_midnight
0,T2,New York,Aer Lingus,May,Tuesday,0,None,No,980.0


In [18]:
newtraindf = pd.get_dummies(traindf)
newtraindf.head(1)

,Minute Delay,Sched_mins_since_midnight,Terminal_T1,Terminal_T2,Destination_Aberdeen,Destination_Abu Dhabi,Destination_Agadir,Destination_Alicante,Destination_Almeria,Destination_Amsterdam,...,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Delay Type_Long,Delay Type_Medium,Delay Type_None,Delay Type_Short,Delay? (15 min)_No,Delay? (15 min)_Yes
0,0,980.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [22]:
# Split Data
x_data = newtraindf.drop(['Minute Delay', 'Delay Type_None', 'Delay Type_Short', 'Delay Type_Long', 'Delay Type_Medium', 'Delay? (15 min)_No', 'Delay? (15 min)_Yes'],axis=1)
y_data = np.vstack((newtraindf['Delay Type_Long'], newtraindf['Delay Type_Medium'], newtraindf['Delay Type_None'], newtraindf['Delay Type_Short'])).T
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2)

n_samples, n_features = x_data.shape
n_outputs = y_data.shape[1]
n_classes = 4

In [25]:
# SGD Regressor, Lasso, Elastic Net, Ridge Regression, SVR(kernel = 'linear'or'rbf'), Ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [27]:
forest = RandomForestClassifier(n_estimators=100)
model = MultiOutputClassifier(forest, n_jobs=-1)
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1)

In [29]:
y_pred = model.predict(X_test)

score2 = metrics.accuracy_score(y_test, y_pred)

print("Accuracy", "", score2)

Accuracy  0.3923566878980892
